In [29]:
from BehaviorAnalyst import BehaviorAnalyst
import pandas as pd
import numpy as np
from tqdm import tqdm
import collections

In [2]:
Dataframe = pd.read_hdf("../DeepLabCutV2/cow-sky-2019-01-27/labeled-data/cow/CollectedData_Sky.h5")

In [4]:
Dataframe.head(5)

scorer                  Sky                                                  \
bodyparts        Body_Cow_A         Body_Cow_B          Body_Cow_C            
coords                    x       y          x        y          x        y   
data/img0000.png     234.24  307.20    764.000  361.333    258.667  686.667   
data/img0250.png     719.36  398.08    441.333  116.000   1240.000  521.333   
data/img0251.png     719.36  399.36    438.667  114.667   1242.667  517.333   
data/img0252.png     719.36  399.36    438.667  114.667   1245.333  518.667   
data/img0253.png     719.36  399.36    438.667  114.667   1254.667  516.000   

scorer                                                                     \
bodyparts        Bottom_Cow_A          Bottom_Cow_B          Bottom_Cow_C   
coords                      x        y            x        y            x   
data/img0000.png       54.400  388.267      571.733  187.733      293.333   
data/img0250.png      987.733  362.667      710.400   70.400     1099.733   
data/img0251.png      987.733  362.667      709.333   70.400     1098.667   
data/img0252.png      987.733  362.667      709.333   72.533     1102.933   
data/img0253.png      987.733  362.667      707.200   69.333     1113.600   

scorer                                                                    \
bodyparts                 Nose_Cow_A          Nose_Cow_B      Nose_Cow_C   
coords                  y          x        y          x    y          x   
data/img0000.png  504.533    645.333  289.067        0.0  0.0     362.24   
data/img0250.png  516.267      0.000    0.000        0.0  0.0       0.00   
data/img0251.png  516.267      0.000    0.000        0.0  0.0       0.00   
data/img0252.png  508.800      0.000    0.000        0.0  0.0       0.00   
data/img0253.png  506.667      0.000    0.000        0.0  0.0       0.00   

scorer                    
bodyparts                 
coords                 y  
data/img0000.png  872.96  
data/img0250.png    0.00  
data/img0251.png    0.00  
data/img0252.png    0.00  
data/img0253.png    0.00

In [5]:
scorer = np.unique(Dataframe.columns.get_level_values(0))[0]
print("Scorer:", scorer)
nframes = len(Dataframe.index)
print("Number of Frames:", nframes)
cows = ['Cow_A', 'Cow_B', 'Cow_C']
bodyparts = ['Nose_', 'Bottom_', 'Body_']
bodyparts2compare = {
    'Nose_': bodyparts,
    'Bottom_': ['Nose_']
}
MARKER_CUTOFF = 1
analyst = BehaviorAnalyst()
features = collections.defaultdict(dict)

Scorer: Sky
Number of Frames: 1300


In [6]:
for cow in cows:
    for index in tqdm(range(nframes)):
        frame_name = Dataframe.index[index]
        feature_container = list()
        for part2compare in bodyparts2compare:
            for pair2compare in bodyparts2compare[part2compare]:
                distances = [np.Infinity]
                for comparative_cow in cows:
                    if comparative_cow != cow:
                        x = Dataframe[scorer][part2compare + cow]['x'].values[index]
                        y = Dataframe[scorer][part2compare + cow]['y'].values[index]
                        x_comparative = Dataframe[scorer][pair2compare + comparative_cow]['x'].values[index]
                        y_comparative = Dataframe[scorer][pair2compare + comparative_cow]['y'].values[index]
                        if (((x >= MARKER_CUTOFF) or (y >= MARKER_CUTOFF)) and ((x_comparative >= MARKER_CUTOFF) or (y_comparative >= MARKER_CUTOFF))):
                            # if all of body parts appear.
                            point = [[x, y]]
                            point_comparative = [[x_comparative, y_comparative]]
                            dist = analyst.calculateDistance(point, point_comparative)
                            distances.append(dist)
#                         else:
#                             distances.append(-np.Infinity)

                feature_container.append(min(distances))
        features[cow][frame_name] = feature_container

100%|██████████| 1300/1300 [00:30<00:00, 40.63it/s]


In [7]:
FEATURE_NAMES = ['nearest_dist_head2head', 'nearest_dist_head2bottom', 'nearest_dist_head2body', 'nearest_dist_bottom2head']
df_cows_features = dict()
for cow in features:
    df_cow_features = collections.defaultdict(dict)
    for index in range(len(FEATURE_NAMES)):
        attribute = list()
        for frame in features[cow]:
            attribute.append(features[cow][frame][index])
        df_cow_features[cow][FEATURE_NAMES[index]] = pd.Series(attribute, index=list(features[cow].keys()))
    df_cows_features[cow] = pd.DataFrame(df_cow_features[cow])

In [19]:
df_cows_features['Cow_A'].head(10)

,nearest_dist_head2head,nearest_dist_head2bottom,nearest_dist_head2body,nearest_dist_bottom2head
data/img0000.png,648.901134,125.241924,138.939669,574.188793
data/img0250.png,inf,inf,inf,inf
data/img0251.png,inf,inf,inf,inf
data/img0252.png,inf,inf,inf,inf
data/img0253.png,inf,inf,inf,inf
data/img0254.png,inf,inf,inf,inf
data/img0255.png,inf,inf,inf,inf
data/img0256.png,inf,inf,inf,inf
data/img0257.png,inf,inf,inf,inf
data/img0258.png,inf,inf,inf,inf


In [32]:
# Save data by this scorer
folder2save = 'extracted_features/'
for cow in df_cows_features:
    df_cows_features[cow].to_csv(folder2save + "ExtractedFeatures_" + scorer + "_" + cow + ".csv")
    df_cows_features[cow].to_hdf(folder2save + "ExtractedFeatures_" + scorer + "_" + cow + '.h5', 'df_with_missing', format='table', mode='w')